In [1]:
#!/usr/bin/env python

import numpy as np, os, sys
from scipy.io import loadmat
#from evaluate_12ECG_score import compute_beta_score, compute_auc
import pandas as pd
from global_vars import headers, labels
    
def load_challenge_data(filename):


    x = loadmat(filename)
    data = np.asarray(x['val'], dtype=np.float64)

    new_file = filename.replace('.mat','.hea')
    input_header_file = os.path.join(new_file)

    with open(input_header_file,'r') as f:
        header_data=f.readlines()


    return data, header_data


def save_challenge_predictions(output_directory,filename,scores,labels,classes):

    recording = os.path.splitext(filename)[0]
    new_file = filename.replace('.mat','.csv')
    output_file = os.path.join(output_directory,new_file)

    # Include the filename as the recording number
    recording_string = '#{}'.format(recording)
    class_string = ','.join(classes)
    label_string = ','.join(str(i) for i in labels)
    score_string = ','.join(str(i) for i in scores)

    with open(output_file, 'w') as f:
        f.write(recording_string + '\n' + class_string + '\n' + label_string + '\n' + score_string + '\n')

  
# Find unique number of classes  
def get_classes(input_directory,files):

    classes=set()
    for f in files:
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    for c in tmp:
                        classes.add(c.strip())

    return sorted(classes)

def collect_data(input_directory, input_files):
    print('Extracting 12ECG features...')
    # Extract features
    num_files = len(input_files)
    datas = []
    header_datas = []
    #features = []
    for i, f in enumerate(input_files):
        print('    {}/{}...'.format(i+1, num_files))
        tmp_input_file = os.path.join(input_directory,f)
        data,header_data = load_challenge_data(tmp_input_file)
        datas.append(data)
        header_datas.append(header_data)
    return datas, header_datas

import pickle

from scipy import signal 

def butter_bandpass(lowcut, highcut, fs, order=5, vis=False):
    nyq = 0.5 * fs # fs / 2
    low = lowcut / nyq # lowcut * 2 / fs
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    # visualize the filter
    if vis:
        w, h = signal.freqz(b, a, worN=2000)
        plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order) # fs / (2 * pi)
        plt.title('Butterworth filter frequency response')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Gain')
        plt.margins(0, 0.1)
        plt.grid(which='both', axis='both')
        plt.axvline(low, color='green') # cutoff frequency
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    # lowcut, fs in Hz
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.filtfilt(b, a, data)
    
    return y

def filter_data(Data, highcut=None):
    Data_filtered = np.zeros(Data.shape)
    chns = Data.shape[0]
    for chn in range(chns):
        if highcut is None:
            filtered_ecg = butter_bandpass_filter(Data[chn,:], lowcut=0.5,
                                                  highcut=30.0, fs=500.0,
                                                  order=4)
            Data_filtered[chn, :] = filtered_ecg
        else:
            filtered_ecg = butter_bandpass_filter(Data[chn,:], lowcut=0.5,
                                                  highcut=highcut, fs=500.0,
                                                  order=4)
            Data_filtered[chn, :] = filtered_ecg            

    return Data_filtered


from scipy.signal import find_peaks
def sep_rr_interval(bspm, percentile, distance=400, period_len=800, plot=False, p='', idx=''):
    # bspm: 252 x t
    
    rr_intervals = []
    total_time = bspm.shape[1]
    lead_vars = np.zeros((total_time,))
    for t in range(total_time):
        lead_vars[t] = np.var(bspm[:,t])
    lead_vars/=np.max(lead_vars)
    
    start_point = int(period_len/2)
    n_periods = int(len(lead_vars)/start_point)
    peak_values = []
    for i in range(n_periods-1):
        
        peak_value_ints = find_peaks(lead_vars[start_point*i:start_point*i+period_len])[0]
        if len(peak_value_ints) > 0:
            peak_value_ints_max = np.max(lead_vars[start_point*i:][peak_value_ints])
            peak_values.append(peak_value_ints_max)
    
    #print(peak_values)
    peak_value = 0
    if len(peak_values) > 0:
        peak_value = np.min(peak_values)
        
    peaks = find_peaks(lead_vars, height=peak_value, distance=distance)[0]
    if plot:
        plt.plot(lead_vars)
    for i in range(len(peaks)-1):
        if plot:
            plt.axvline(peaks[i], c='C1')
        rr_intervals.append((peaks[i], peaks[i+1]))
    intervals = np.diff(peaks)
    mean_intervals = np.mean(intervals)
    std_intervals = np.std(intervals)
    if plot:
        if len(peaks) > 0:
            plt.axvline(peaks[-1], c='C1')
        plt.axhline(peak_value, c='C2', label=str(peak_value))
        plt.legend()
        plt.title(str(len(intervals))+' ' + str(mean_intervals) + '_' + str(std_intervals))
        plt.savefig('../result/pAF/rr/'+str(idx) + '_'+str(p))
        
        plt.close()
    return rr_intervals, mean_intervals, std_intervals

        
def get_basic_info(header_data, labels):
    tmp_hea = header_data[0].split(' ')
    ptID = tmp_hea[0]
    age = None
    sex = None
    label = None
    for iline in header_data:
        if iline.startswith('#Age'):
            tmp_age = iline.split(': ')[1].strip()
            age = int(tmp_age if tmp_age != 'NaN' else 57)
        elif iline.startswith('#Sex'):
            tmp_sex = iline.split(': ')[1]
            if tmp_sex.strip()=='Female':
                sex =1
            else:
                sex=0
        elif iline.startswith('#Dx'):
            Dx = np.char.rstrip(iline.split(': ')[1].split(','))
            label = [ll in Dx for ll in labels]
            
    
    return [ptID, sex, age] + label

In [2]:
print('Load data...')

if not 'datas' in locals() or datas is None:
    datas = None
    header_datas = None
    classes = None
    with open('../saved/datas.pkl', 'rb') as f:
        datas = pickle.load(f)
    with open('../saved/header_datas.pkl', 'rb') as f:
        header_datas = pickle.load(f)
    with open('../saved/classes.pkl', 'rb') as f:
        classes = pickle.load(f)

Load data...


In [3]:
classes

['AF', 'I-AVB', 'LBBB', 'Normal', 'PAC', 'PVC', 'RBBB', 'STD', 'STE']

In [11]:
n_samples = [int(header_data[0].split(' ')[3]) for header_data in header_datas]

In [12]:
np.min(n_samples)

3000

In [13]:
Hzs = [int(header_data[0].split(' ')[2]) for header_data in header_datas]

In [14]:
import matplotlib.pyplot as plt
fDatas = []
infos =[]
period_len = 400
rr_max = 650
std_th = 220
rr_min = 200
plot = False
for idx in range(len(datas)):
    # basic info
    info = get_basic_info(header_datas[idx], labels)


    # get data
    ptID = info[0]


    fData = filter_data(datas[idx][:,2000:], highcut=50.0)
    #fData = datas[idx][:,1000:]
    intervals, mean_intervals, std_intervals = sep_rr_interval(fData[6:], percentile=90, distance=rr_min, 
                                plot=plot, idx=idx, p=ptID, period_len=period_len)
    if len(intervals) == 0 or mean_intervals > rr_max or std_intervals > std_th:
        intervals, mean_intervals, std_intervals = sep_rr_interval(fData[:6], percentile=90, distance=rr_min, 
                                plot=plot, idx=idx, p=ptID, period_len=period_len)
    if len(intervals) == 0 or mean_intervals > rr_max or std_intervals > std_th:
        intervals, mean_intervals, std_intervals = sep_rr_interval(fData[:3], percentile=90, distance=rr_min, 
                                plot=plot, idx=idx, p=ptID, period_len=period_len)  
    info += [mean_intervals, std_intervals]
    print(str(idx) + ' ' + ptID + " {:.2f} {:.2f}".format(mean_intervals, std_intervals))

    if len(intervals) == 0 or mean_intervals > rr_max or std_intervals > std_th:
        break



    for i in range(len(intervals)):
        l, r = intervals[i]
        fDatas.append(fData[:,l:r])
        infos.append(info + [r-l]) # add interval length

    #     plt.plot(fData[0,l-100:r+100])
    #     plt.savefig('../result/pAF/seg/'+str(idx)+'_'+ptID+'_'+str(i))
    #     plt.close()

0 A0001 298.94 2.57
1 A0002 381.71 8.89
2 A0003 284.38 76.61
3 A0004 285.46 61.39
4 A0005 278.94 79.05
5 A0006 442.80 59.48
6 A0007 251.91 6.65
7 A0008 412.38 79.48
8 A0009 339.47 133.11
9 A0010 422.29 12.58
10 A0011 435.11 41.14
11 A0012 405.93 120.43
12 A0013 433.86 25.41
13 A0014 433.08 85.18
14 A0015 411.52 15.70
15 A0016 422.44 18.03
16 A0017 349.71 57.30
17 A0018 326.69 1.12
18 A0019 334.80 82.17
19 A0020 426.93 13.35
20 A0021 448.17 10.01
21 A0022 381.72 12.82
22 A0023 353.88 98.40
23 A0024 272.27 63.75
24 A0025 303.78 61.50
25 A0026 451.14 55.99
26 A0027 250.00 0.60
27 A0028 356.00 3.25
28 A0029 415.14 18.18
29 A0030 430.89 108.14
30 A0031 329.88 2.59
31 A0032 498.13 41.91
32 A0033 291.60 77.33
33 A0034 591.50 9.01
34 A0035 462.57 3.81
35 A0036 364.78 155.89
36 A0037 436.00 6.51
37 A0038 436.00 10.07
38 A0039 435.17 7.22
39 A0040 307.74 13.20
40 A0041 451.17 49.57
41 A0042 409.38 5.61
42 A0043 353.00 69.01
43 A0044 349.29 2.66
44 A0045 421.67 118.79
45 A0046 371.31 151.50
46 A0

364 A0365 408.77 20.19
365 A0366 432.80 4.60
366 A0367 328.69 68.05
367 A0368 427.17 2.48
368 A0369 468.90 19.51
369 A0370 348.62 117.99
370 A0371 415.16 38.89
371 A0372 407.90 44.67
372 A0373 286.48 105.66
373 A0374 295.48 74.29
374 A0375 324.33 84.67
375 A0376 336.09 88.59
376 A0377 324.24 125.51
377 A0378 372.25 65.83
378 A0379 303.67 85.70
379 A0380 336.74 64.20
380 A0381 408.38 5.26
381 A0382 475.20 3.97
382 A0383 416.85 100.35
383 A0384 263.90 28.89
384 A0385 326.00 96.30
385 A0386 370.58 11.16
386 A0387 383.91 13.74
387 A0388 285.41 4.43
388 A0389 422.13 109.21
389 A0390 309.78 68.54
390 A0391 378.86 8.84
391 A0392 340.00 85.86
392 A0393 388.57 14.86
393 A0394 363.29 85.51
394 A0395 375.14 7.08
395 A0396 354.25 3.03
396 A0397 277.92 3.64
397 A0398 442.67 6.05
398 A0399 299.78 41.58
399 A0400 476.86 7.14
400 A0401 457.62 3.08
401 A0402 440.50 9.91
402 A0403 426.08 35.70
403 A0404 342.33 4.83
404 A0405 372.57 82.42
405 A0406 647.25 89.49
406 A0407 393.57 19.26
407 A0408 399.57 16.

722 A0723 399.71 103.01
723 A0724 362.00 78.29
724 A0725 432.33 7.13
725 A0726 416.62 50.42
726 A0727 386.00 2.33
727 A0728 491.00 11.07
728 A0729 469.09 3.82
729 A0730 461.00 7.23
730 A0731 426.82 86.43
731 A0732 384.71 8.48
732 A0733 358.14 112.47
733 A0734 384.33 2.11
734 A0735 320.80 7.93
735 A0736 309.00 0.94
736 A0737 343.94 65.22
737 A0738 292.32 9.24
738 A0739 394.19 36.36
739 A0740 464.50 187.56
740 A0741 325.18 6.46
741 A0742 384.14 112.58
742 A0743 396.43 15.01
743 A0744 358.00 58.98
744 A0745 356.15 110.09
745 A0746 388.50 121.85
746 A0747 391.86 73.39
747 A0748 470.45 42.03
748 A0749 444.22 20.45
749 A0750 356.53 96.62
750 A0751 301.38 57.86
751 A0752 364.47 74.55
752 A0753 436.70 8.47
753 A0754 354.69 4.56
754 A0755 357.90 17.54
755 A0756 407.21 27.07
756 A0757 430.39 76.74
757 A0758 343.64 12.34
758 A0759 384.14 8.90
759 A0760 357.12 116.10
760 A0761 445.33 16.18
761 A0762 326.00 86.84
762 A0763 300.40 46.79
763 A0764 431.83 72.12
764 A0765 329.88 2.15
765 A0766 368.00 1

1079 A1080 367.56 11.97
1080 A1081 429.50 46.21
1081 A1082 420.83 3.76
1082 A1083 407.58 73.56
1083 A1084 508.29 5.65
1084 A1085 360.75 149.83
1085 A1086 310.00 5.08
1086 A1087 360.29 112.57
1087 A1088 312.50 2.60
1088 A1089 382.29 45.09
1089 A1090 383.00 9.27
1090 A1091 411.83 3.08
1091 A1092 427.88 7.73
1092 A1093 373.42 115.07
1093 A1094 281.64 80.25
1094 A1095 336.08 8.20
1095 A1096 379.36 72.50
1096 A1097 436.33 7.93
1097 A1098 390.33 95.88
1098 A1099 464.57 5.88
1099 A1100 312.10 63.31
1100 A1101 408.33 115.04
1101 A1102 296.12 3.89
1102 A1103 368.71 114.94
1103 A1104 384.33 7.20
1104 A1105 363.00 2.39
1105 A1106 435.15 14.72
1106 A1107 283.30 3.95
1107 A1108 310.71 129.02
1108 A1109 404.55 85.12
1109 A1110 310.75 76.42
1110 A1111 251.54 1.82
1111 A1112 459.00 7.00
1112 A1113 406.00 13.33
1113 A1114 401.50 82.57
1114 A1115 470.50 9.22
1115 A1116 562.00 8.65
1116 A1117 407.67 7.48
1117 A1118 426.44 19.78
1118 A1119 356.57 4.40
1119 A1120 353.17 9.12
1120 A1121 395.62 8.92
1121 A11

1425 A1426 321.55 1.72
1426 A1427 341.08 58.50
1427 A1428 395.86 105.87
1428 A1429 364.45 43.84
1429 A1430 346.38 3.29
1430 A1431 349.50 11.05
1431 A1432 279.25 73.34
1432 A1433 404.25 3.96
1433 A1434 407.50 2.87
1434 A1435 368.43 2.97
1435 A1436 367.00 52.40
1436 A1437 309.40 1.36
1437 A1438 324.27 80.87
1438 A1439 276.70 64.72
1439 A1440 295.36 70.59
1440 A1441 336.13 4.16
1441 A1442 510.50 41.56
1442 A1443 515.80 9.81
1443 A1444 314.86 42.05
1444 A1445 406.89 9.17
1445 A1446 366.54 96.96
1446 A1447 444.67 11.69
1447 A1448 443.67 49.72
1448 A1449 449.00 62.66
1449 A1450 362.00 100.24
1450 A1451 390.22 8.78
1451 A1452 349.00 136.58
1452 A1453 389.43 62.66
1453 A1454 317.13 47.85
1454 A1455 388.75 22.21
1455 A1456 481.00 32.47
1456 A1457 313.50 115.18
1457 A1458 301.17 102.51
1458 A1459 420.17 5.87
1459 A1460 331.41 115.17
1460 A1461 605.50 2.87
1461 A1462 271.78 63.01
1462 A1463 371.88 10.33
1463 A1464 409.93 112.08
1464 A1465 382.54 10.81
1465 A1466 444.10 10.47
1466 A1467 290.33 63.

...

5466 A5467 322.73 82.88
5467 A5468 493.50 16.63
5468 A5469 379.06 120.64
5469 A5470 398.14 24.36
5470 A5471 418.37 4.72
5471 A5472 288.33 5.57
5472 A5473 387.36 6.59
5473 A5474 368.59 4.32
5474 A5475 340.10 1.51
5475 A5476 390.81 10.09
5476 A5477 486.03 14.31
5477 A5478 399.00 6.78
5478 A5479 345.27 5.50
5479 A5480 471.04 21.26
5480 A5481 393.60 15.23
5481 A5482 481.70 18.42
5482 A5483 396.67 3.82
5483 A5484 373.38 17.15
5484 A5485 390.22 64.05
5485 A5486 413.07 6.59
5486 A5487 437.00 4.17
5487 A5488 301.42 81.29
5488 A5489 428.17 153.87
5489 A5490 447.50 8.28
5490 A5491 472.40 10.40
5491 A5492 469.91 7.23
5492 A5493 490.80 18.10
5493 A5494 287.36 41.25
5494 A5495 412.29 5.28
5495 A5496 343.75 146.64
5496 A5497 257.90 87.40
5497 A5498 323.54 3.42
5498 A5499 442.25 63.01
5499 A5500 494.18 6.00
5500 A5501 404.00 13.93
5501 A5502 357.48 75.90
5502 A5503 342.87 1.71
5503 A5504 452.50 6.55
5504 A5505 448.88 6.01
5505 A5506 479.44 44.33
5506 A5507 446.56 14.18
5507 A5508 301.00 55.23
5508 A5

5814 A5815 288.89 104.63
5815 A5816 462.18 57.00
5816 A5817 315.60 79.68
5817 A5818 334.93 106.21
5818 A5819 294.50 63.24
5819 A5820 362.34 86.27
5820 A5821 237.24 41.90
5821 A5822 363.57 5.83
5822 A5823 345.82 82.63
5823 A5824 351.56 4.67
5824 A5825 474.75 14.75
5825 A5826 468.54 17.05
5826 A5827 346.50 7.62
5827 A5828 278.62 59.94
5828 A5829 279.19 74.11
5829 A5830 308.00 7.83
5830 A5831 377.29 13.40
5831 A5832 398.90 65.79
5832 A5833 455.50 56.85
5833 A5834 460.67 111.16
5834 A5835 446.83 127.63
5835 A5836 353.43 2.06
5836 A5837 318.44 84.40
5837 A5838 385.77 17.46
5838 A5839 281.78 71.49
5839 A5840 378.00 6.08
5840 A5841 382.10 94.52
5841 A5842 326.31 66.81
5842 A5843 491.29 4.59
5843 A5844 293.89 2.18
5844 A5845 430.10 98.18
5845 A5846 349.64 102.78
5846 A5847 374.73 8.05
5847 A5848 515.20 14.88
5848 A5849 409.83 43.41
5849 A5850 310.62 9.55
5850 A5851 406.00 4.94
5851 A5852 407.30 10.65
5852 A5853 428.55 50.01
5853 A5854 351.17 114.34
5854 A5855 503.44 35.24
5855 A5856 338.29 115

6160 A6161 417.50 93.70
6161 A6162 434.00 7.91
6162 A6163 333.47 9.76
6163 A6164 346.33 57.10
6164 A6165 389.80 84.51
6165 A6166 406.54 32.74
6166 A6167 411.46 6.42
6167 A6168 447.00 6.08
6168 A6169 311.20 10.98
6169 A6170 336.31 55.31
6170 A6171 314.94 5.74
6171 A6172 338.44 50.26
6172 A6173 416.71 59.24
6173 A6174 301.28 50.55
6174 A6175 456.27 4.24
6175 A6176 423.14 80.94
6176 A6177 343.10 104.44
6177 A6178 425.12 39.95
6178 A6179 417.83 7.54
6179 A6180 297.20 79.69
6180 A6181 354.90 122.09
6181 A6182 365.62 46.90
6182 A6183 412.74 5.75
6183 A6184 503.20 68.05
6184 A6185 303.11 33.98
6185 A6186 293.11 42.83
6186 A6187 329.52 87.77
6187 A6188 401.69 16.97
6188 A6189 391.71 4.92
6189 A6190 304.22 1.55
6190 A6191 369.29 134.59
6191 A6192 419.13 19.12
6192 A6193 298.78 2.25
6193 A6194 278.30 85.35
6194 A6195 418.00 8.41
6195 A6196 395.36 23.65
6196 A6197 388.12 161.97
6197 A6198 318.00 16.11
6198 A6199 481.67 31.18
6199 A6200 380.00 7.48
6200 A6201 258.71 62.15
6201 A6202 401.23 148.81


6508 A6509 315.07 37.08
6509 A6510 436.50 39.84
6510 A6511 461.00 16.61
6511 A6512 371.00 91.57
6512 A6513 446.50 46.55
6513 A6514 316.44 2.99
6514 A6515 342.39 75.12
6515 A6516 350.86 2.53
6516 A6517 385.86 112.89
6517 A6518 359.10 4.55
6518 A6519 272.32 77.88
6519 A6520 301.12 62.75
6520 A6521 413.33 4.07
6521 A6522 398.56 13.56
6522 A6523 410.17 75.10
6523 A6524 423.67 4.50
6524 A6525 258.71 62.15
6525 A6526 484.30 39.97
6526 A6527 345.22 50.71
6527 A6528 389.43 103.01
6528 A6529 491.23 19.82
6529 A6530 480.25 3.11
6530 A6531 327.88 12.29
6531 A6532 363.21 36.00
6532 A6533 506.78 3.58
6533 A6534 398.71 15.14
6534 A6535 394.62 10.22
6535 A6536 482.50 1.98
6536 A6537 336.17 85.79
6537 A6538 276.22 0.42
6538 A6539 339.50 61.57
6539 A6540 422.86 15.06
6540 A6541 362.50 10.25
6541 A6542 449.65 34.42
6542 A6543 296.08 5.24
6543 A6544 522.80 57.42
6544 A6545 376.55 59.53
6545 A6546 332.00 109.11
6546 A6547 404.40 12.63
6547 A6548 384.29 129.72
6548 A6549 364.67 79.79
6549 A6550 398.00 41.4

6856 A6867 373.67 66.11
6857 A6868 346.64 132.47
6858 A6869 447.33 91.69
6859 A6870 396.86 4.52
6860 A6871 321.44 6.45
6861 A6872 452.00 67.15
6862 A6873 420.09 16.29
6863 A6874 505.45 8.91
6864 A6875 295.22 4.59
6865 A6876 293.75 48.98
6866 A6877 387.64 127.24
6867 A6804 394.71 128.16
6868 A6805 413.65 25.79
6869 A6806 480.79 8.86
6870 A6807 305.40 47.89
6871 A6808 290.18 39.47
6872 A6809 408.29 4.56
6873 A6810 390.52 4.97
6874 A6811 511.71 95.28
6875 A6812 269.80 1.33
6876 A6813 511.15 20.55


In [ ]:
df_infos = pd.DataFrame(infos, columns=['ptID','Sex','Age']+labels+['mean_intervals','std_intervals','interval_len'])
df_infos.to_csv('../saved/infos.csv')

In [10]:
fDataRes=np.array([signal.resample(fData, 250, axis=1) for fData in fDatas])
np.save('../saved/fDataRes50Hz.npy', fDataRes)
#[plt.plot(fDataRe[6]) for fDataRe in fDataRes]

In [13]:
from sklearn.preprocessing import normalize
#fDataRes = np.load('../saved/fDataRes50Hz.npy')
fDataReNorms=[fDataRe-np.tile(fDataRe[:,0],(250,1)).transpose() for fDataRe in fDataRes]
fDataReNorms=np.array([normalize(fDataRe) for fDataRe in fDataRes])
np.save('../saved/fDataReNorms50Hz.npy', fDataReNorms)

```matlab
function helperCreateRGBfromTF(ECGData,parentFolder,childFolder)
% This function is only intended to support the ECGAndDeepLearningExample.
% It may change or be removed in a future release.

imageRoot = fullfile(parentFolder,childFolder);

data = ECGData.Data;
labels = ECGData.Labels;

[~,signalLength] = size(data);

fb = cwtfilterbank('SignalLength',signalLength,'VoicesPerOctave',12);
r = size(data,1);

for ii = 1:r
    cfs = abs(fb.wt(data(ii,:)));
    im = ind2rgb(im2uint8(rescale(cfs)),jet(128));
    
    imgLoc = fullfile(imageRoot,char(labels(ii)));
    imFileName = strcat(char(labels(ii)),'_',num2str(ii),'.jpg');
    imwrite(imresize(im,[224 224]),fullfile(imgLoc,imFileName));
end
end
```